# Define variables

We can move the variables that won't change to the `workflow_executor`

In [ ]:
import time
import os

from src import workflow_executor

In [ ]:
# Cloud functions must be deployed to retrieve it name
# The name of the cloud function must be "af-update-experiment-status"
function_metadata = ! gcloud functions describe af-update-experiment-status --gen2 --region us-central1 --format="value(serviceConfig.uri)"
function_uri = function_metadata[0]

### Project, GCS and job ID

In [ ]:
project_id = 'rl-alphafold-dev'     # Your project ID
bucket_name = 'rl-alphafold-dbs'    # Bucket name
region = 'us-central1'              # Region

local_sequence_path = 'sequences/seq1.fasta'    # Path to your local sequence
image_uri = f'gcr.io/{project_id}/alphafold'    # Path to image on GCR

In [ ]:
# Don't change
with open(local_sequence_path, 'r') as fp:
    sequence_str = fp.read()

job_id = f'job-alphafold-{int(time.time()*1000)}'
blob_sequence_path = os.path.join(job_id, 'sequence.fasta')
gcs_job_path = os.path.join(bucket_name, job_id)

In [ ]:
# Change to the desired machine type
data_pipeline_machine_type = 'c2-standard-16'
predict_relax_machine_type = 'n1-standard-8-t4'

### Network

In [ ]:
network_name = 'alphafold'          # Change to your network name
subnetwork_name = 'alphafold-nw'    # Change to your subnetwork name

nfs_ip_address = '10.130.0.2'       # Filestore IP
nfs_path = '/datasets'              # Path to NFS share

In [ ]:
# Don't change
network = f'projects/{project_id}/global/networks/{network_name}'
subnetwork = f'projects/{project_id}/regions/{region}/subnetworks/{subnetwork_name}'

### Data Pipeline

In [ ]:
max_template_date = '2020-05-14'
model_preset = 'monomer'        
db_preset = 'reduced_dbs'       

In [ ]:
# Don't change
data_metadata_filename = 'data_pipeline.json'
msas_folder = 'msas'
data_features_filename = 'features.pkl'

blob_data_metadata_path = os.path.join(job_id,'data_pipeline.json')
blob_data_features_path = os.path.join(job_id,'features.pkl')

### Predict and Relax

In [ ]:
# Change to the location (parent path) where the parameters are stored (GCS)
model_params_path = 'rl-alphafold-dbs'

In [ ]:
# Change only if necessary
num_predictions_per_model = 1
run_relax = True
random_seed = 1000
parallelism = 5
num_models = 5

# Execute Workflow

In [ ]:
args = workflow_executor.prepare_args_for_experiments(
    project_id,
    bucket_name,
    region,
    function_uri,
    data_pipeline_machine_type,
    predict_relax_machine_type,
    sequence_str,
    job_id,
    blob_sequence_path,
    gcs_job_path,
    image_uri,
    network_name,
    subnetwork_name,
    nfs_ip_address,
    nfs_path,
    network,
    subnetwork,
    max_template_date,
    model_preset,
    db_preset,
    data_metadata_filename,
    msas_folder,
    data_features_filename,
    blob_data_metadata_path,
    blob_data_features_path,
    num_predictions_per_model,
    model_params_path,
    num_models,
    run_relax,
    random_seed,
    parallelism
)

In [ ]:
workflow_executor.execute_workflow('af-batch', args)